# Kazi ya uainishaji wa maandishi

Kama tulivyotaja, tutazingatia kazi rahisi ya uainishaji wa maandishi kwa kutumia seti ya data ya **AG_NEWS**, ambayo ni kuainisha vichwa vya habari vya habari katika mojawapo ya makundi 4: Dunia, Michezo, Biashara, na Sayansi/Teknolojia.

## Seti ya Data

Seti hii ya data imejumuishwa ndani ya moduli ya [`torchtext`](https://github.com/pytorch/text), kwa hivyo tunaweza kuipata kwa urahisi.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Hapa, `train_dataset` na `test_dataset` zina makusanyo yanayorudisha jozi za lebo (nambari ya darasa) na maandishi mtawalia, kwa mfano:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Kwa hivyo, wacha tuchapishe vichwa vya habari 10 vya kwanza kutoka kwenye seti yetu ya data:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Kwa sababu seti za data ni iterators, ikiwa tunataka kutumia data mara nyingi tunahitaji kuibadilisha kuwa orodha:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Uwekaji Tokeni

Sasa tunahitaji kubadilisha maandishi kuwa **nambari** ambazo zinaweza kuwakilishwa kama tensors. Ikiwa tunataka uwakilishi wa kiwango cha maneno, tunahitaji kufanya mambo mawili:
* tumia **tokenizer** kugawanya maandishi katika **tokeni**
* tengeneza **msamiati** wa tokeni hizo.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Kutumia msamiati, tunaweza kwa urahisi kuweka msimbo mfululizo wetu uliotenganishwa kuwa seti ya nambari:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Uwakilishi wa maandishi kwa kutumia Bag of Words

Kwa sababu maneno yanawakilisha maana, wakati mwingine tunaweza kuelewa maana ya maandishi kwa kuangalia tu maneno ya kibinafsi, bila kujali mpangilio wake katika sentensi. Kwa mfano, wakati wa kuainisha habari, maneno kama *hali ya hewa*, *theluji* yanaweza kuashiria *utabiri wa hali ya hewa*, wakati maneno kama *hisa*, *dola* yanaweza kuelekea kwenye *habari za kifedha*.

**Bag of Words** (BoW) ni uwakilishi wa vector unaotumika sana katika mbinu za jadi. Kila neno linaunganishwa na faharasa ya vector, na kipengele cha vector kinaonyesha idadi ya mara neno fulani linavyotokea katika hati fulani.

![Picha inayoonyesha jinsi uwakilishi wa vector wa Bag of Words unavyohifadhiwa kwenye kumbukumbu.](../../../../../translated_images/sw/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Unaweza pia kufikiria BoW kama jumla ya vectors zote zilizowakilishwa kwa njia ya one-hot-encoding kwa maneno ya kibinafsi katika maandishi.

Hapa chini kuna mfano wa jinsi ya kuunda uwakilishi wa Bag of Words kwa kutumia maktaba ya python ya Scikit Learn:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Ili kuhesabu vector ya bag-of-words kutoka kwa uwakilishi wa vector wa seti yetu ya data ya AG_NEWS, tunaweza kutumia kazi ifuatayo:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Kumbuka:** Hapa tunatumia variable ya kimataifa `vocab_size` kuonyesha ukubwa wa kawaida wa msamiati. Kwa kuwa mara nyingi ukubwa wa msamiati ni mkubwa sana, tunaweza kupunguza ukubwa wa msamiati kwa maneno yanayotumika mara nyingi zaidi. Jaribu kupunguza thamani ya `vocab_size` na kuendesha msimbo hapa chini, na uone jinsi inavyoathiri usahihi. Unapaswa kutarajia kupungua kwa usahihi, lakini si kwa kiasi kikubwa, badala ya utendaji bora.


## Kufundisha Kainishaji wa BoW

Sasa kwa kuwa tumejifunza jinsi ya kujenga uwakilishi wa Bag-of-Words wa maandishi yetu, hebu tufundishe kainishaji juu yake. Kwanza, tunahitaji kubadilisha seti yetu ya data kwa mafunzo kwa njia ambayo uwakilishi wote wa vekta za nafasi hubadilishwa kuwa uwakilishi wa Bag-of-Words. Hili linaweza kufanyika kwa kupitisha kazi ya `bowify` kama parameter ya `collate_fn` kwa `DataLoader` ya kawaida ya torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Sasa hebu tuelezee mtandao wa neva wa kuainisha rahisi ambao una safu moja ya mstari. Ukubwa wa vector ya ingizo ni sawa na `vocab_size`, na ukubwa wa matokeo unalingana na idadi ya madarasa (4). Kwa sababu tunatatua kazi ya uainishaji, kazi ya mwisho ya uanzishaji ni `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Sasa tutafafanua mzunguko wa mafunzo wa kawaida wa PyTorch. Kwa sababu seti yetu ya data ni kubwa sana, kwa madhumuni ya kufundisha tutafanya mafunzo kwa kipindi kimoja tu, na wakati mwingine hata kwa chini ya kipindi kimoja (kuweka kipengele cha `epoch_size` huturuhusu kupunguza mafunzo). Pia tutaripoti usahihi wa mafunzo uliokusanywa wakati wa mafunzo; mara kwa mara ya kuripoti inabainishwa kwa kutumia kipengele cha `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams na N-Grams

Kikwazo kimoja cha mbinu ya mfuko wa maneno ni kwamba baadhi ya maneno ni sehemu ya misemo ya maneno mengi, kwa mfano, neno 'hot dog' lina maana tofauti kabisa kuliko maneno 'hot' na 'dog' katika muktadha mwingine. Ikiwa tutawakilisha maneno 'hot' na 'dog' kila mara kwa vekta zile zile, inaweza kuchanganya modeli yetu.

Ili kushughulikia hili, **uwakilishi wa N-gram** mara nyingi hutumika katika mbinu za uainishaji wa nyaraka, ambapo marudio ya kila neno, maneno mawili au maneno matatu ni kipengele muhimu kwa kufundisha viainishi. Katika uwakilishi wa bigram, kwa mfano, tutaongeza jozi zote za maneno kwenye msamiati, pamoja na maneno ya awali.

Hapa chini ni mfano wa jinsi ya kuzalisha uwakilishi wa mfuko wa maneno wa bigram kwa kutumia Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Hasara kuu ya mbinu ya N-gram ni kwamba ukubwa wa msamiati huanza kuongezeka kwa kasi sana. Kwa vitendo, tunahitaji kuchanganya uwakilishi wa N-gram na mbinu fulani za kupunguza vipimo, kama vile *embeddings*, ambazo tutajadili katika kipengele kinachofuata.

Ili kutumia uwakilishi wa N-gram katika seti yetu ya data ya **AG News**, tunahitaji kujenga msamiati maalum wa ngram:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Tunaweza kutumia msimbo ule ule kama hapo juu kufundisha classifier, hata hivyo, hii itakuwa na matumizi makubwa ya kumbukumbu. Katika sehemu inayofuata, tutafundisha classifier ya bigram kwa kutumia embeddings.

> **Note:** Unaweza kuacha tu zile ngrams ambazo zinatokea kwenye maandishi zaidi ya idadi maalum ya mara. Hii itahakikisha kwamba bigrams zisizotokea mara nyingi zitaachwa, na itapunguza ukubwa wa vipimo kwa kiasi kikubwa. Ili kufanya hivyo, weka parameter `min_freq` kwa thamani ya juu zaidi, na angalia mabadiliko ya urefu wa msamiati.


## Mzunguko wa Neno na Upekee wa Hati TF-IDF

Katika uwakilishi wa BoW, matukio ya maneno yanapewa uzito sawa, bila kujali neno lenyewe. Hata hivyo, ni wazi kwamba maneno yanayojirudia mara kwa mara, kama *a*, *in*, nk. yana umuhimu mdogo sana kwa uainishaji, ikilinganishwa na maneno maalum. Kwa kweli, katika kazi nyingi za NLP, baadhi ya maneno ni muhimu zaidi kuliko mengine.

**TF-IDF** inasimama kwa **mzunguko wa neno–upekee wa hati**. Ni toleo la begi la maneno, ambapo badala ya thamani ya 0/1 ya binary inayoonyesha uwepo wa neno katika hati, thamani ya namba inayozunguka hutumika, ambayo inahusiana na mzunguko wa matukio ya neno katika mkusanyiko.

Kwa ufafanuzi rasmi zaidi, uzito $w_{ij}$ wa neno $i$ katika hati $j$ hufafanuliwa kama:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ambapo
* $tf_{ij}$ ni idadi ya matukio ya $i$ katika $j$, yaani thamani ya BoW tuliyoiona awali
* $N$ ni idadi ya hati katika mkusanyiko
* $df_i$ ni idadi ya hati zinazojumuisha neno $i$ katika mkusanyiko mzima

Thamani ya TF-IDF $w_{ij}$ huongezeka kulingana na idadi ya mara neno linavyoonekana katika hati na hupunguzwa na idadi ya hati katika mkusanyiko zinazojumuisha neno hilo, jambo ambalo husaidia kurekebisha ukweli kwamba baadhi ya maneno huonekana mara nyingi zaidi kuliko mengine. Kwa mfano, ikiwa neno linaonekana katika *kila* hati katika mkusanyiko, $df_i=N$, na $w_{ij}=0$, na maneno hayo yatapuuzwa kabisa.

Unaweza kuunda kwa urahisi vectorization ya TF-IDF ya maandishi ukitumia Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Hitimisho

Hata hivyo, ingawa uwakilishi wa TF-IDF unatoa uzito wa marudio kwa maneno tofauti, hauwezi kuwakilisha maana au mpangilio. Kama mwanaisimu maarufu J. R. Firth alivyosema mwaka 1935, “Maana kamili ya neno daima ni ya muktadha, na hakuna utafiti wa maana bila muktadha unaoweza kuchukuliwa kwa uzito.”. Tutajifunza baadaye katika kozi jinsi ya kunasa taarifa za muktadha kutoka kwa maandishi kwa kutumia uundaji wa lugha.



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati asilia katika lugha yake ya awali inapaswa kuchukuliwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
